In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\Text-Summerization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ARSH\\Desktop\\MLOps\\Text-Summerization'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    metrics_file: Path
    tokenizer_path: Path

In [18]:
from textSummaizer.constants import *
from textSummaizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            metrics_file = config.metrics_file,
            tokenizer_path =config.tokenizer_path
        )

        return model_evaluation_config

In [20]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [24]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)

    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i + batch_size]
            

    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def eval(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_t5 = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:100],
            rouge_metric, 
            model_t5,
            tokenizer, 
            batch_size=2, 
            column_text='dialogue', 
            column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
        df = pd.DataFrame(rouge_dict, index=['t5-small']) 
        df.to_csv(self.config.metrics_file, index=False)

    

In [25]:

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.eval()
except Exception as e:
    raise e

[2025-07-29 19:33:20,018 - textSummarizerlogger - INFO - YAML file config\config.yaml read successfully.]
[2025-07-29 19:33:20,019 - textSummarizerlogger - INFO - YAML file params.yaml read successfully.]
[2025-07-29 19:33:20,021 - textSummarizerlogger - INFO - Created directory: artifacts]
[2025-07-29 19:33:20,022 - textSummarizerlogger - INFO - Created directory: artifacts/model_evaluation]
[2025-07-29 19:33:20,019 - textSummarizerlogger - INFO - YAML file params.yaml read successfully.]
[2025-07-29 19:33:20,021 - textSummarizerlogger - INFO - Created directory: artifacts]
[2025-07-29 19:33:20,022 - textSummarizerlogger - INFO - Created directory: artifacts/model_evaluation]


100%|██████████| 50/50 [03:39<00:00,  4.40s/it]

[2025-07-29 19:37:02,437 - absl - INFO - Using default tokenizer.]
